# Mode shapes using Edge detection

In [ ]:
#Edge detection using Canny edge detection And ROI
import cv2
import numpy as np

file = "filename"

# SHOW FIRST FRAME OF VIDEO        
def get_frame(filename,index):
    counter=0
    video=cv2.VideoCapture(filename)
    while video.isOpened():
        rete,frame=video.read()
        if rete:
            if counter==index:
                return frame
            counter +=1
        else:
            break
    video.release()
    return None  

def nothing(x):
    pass

cv2.namedWindow('Canny')

cv2.createTrackbar('lower', 'Canny', 0, 255, nothing) 
cv2.createTrackbar('upper', 'Canny', 1, 255, nothing)
cv2.createTrackbar('blur','Canny',1,21,nothing)

frame = get_frame(file,0) 
# Select ROI
roi = cv2.selectROI('Region of interest', frame)

# Create ROI coordinates
x, y = roi[0], roi[1]
w,h = roi[2], roi[3]
ROI = frame[y:y+h, x:x+w]

while True:
    frame = frame
    lower = cv2.getTrackbarPos('lower', 'Canny')
    upper = cv2.getTrackbarPos('upper', 'Canny')
    blur = 2*cv2.getTrackbarPos('blur', 'Canny')-1 # Må være oddetall
    
    
    Gauss = cv2.GaussianBlur(ROI,(blur,blur),cv2.BORDER_DEFAULT)
    edges = cv2.Canny(Gauss,lower,upper)
    
    cv2.imshow('Canny',edges)
    
    key = cv2.waitKey(1)
    if key == 27:
        break 

cv2.destroyAllWindows();
cv2.waitKey(1)

# Canny edge detection on full video
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Video to track
cap = cv2.VideoCapture(file)
Edges = []
fps = int(cap.get(cv2.CAP_PROP_FPS))
frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

coord = []
while True:
    ret, frame = cap.read()
    if not ret:
        print('Video Finished')
        break 
    
    #Region of interest
    ROI = frame[y:y+h, x:x+w]
    height, width, channels = frame.shape
    
    # Find edges with Gaussian blur and limits for hysteresis
    Gauss = cv2.GaussianBlur(ROI,(blur,blur),cv2.BORDER_DEFAULT)
    edges = cv2.Canny(Gauss,lower,upper)
    
    # Saving
    indices = np.where(edges != [0])
    coordinates = (zip(indices[0], indices[1]))
    coord.append(list(coordinates))
    
    # Finding contours of edges to draw in the video
    contours = cv2.findContours(edges,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    cont = cv2.drawContours(ROI, contours[0], -1, (0,255,0), thickness = 2)
    Edges.append(cont) 
    
    #cv2.imshow("Edges", cont)
    frame[y:y+h, x:x+w] = cont
    
    cv2.imshow('blur', Gauss)
    cv2.imshow('image', frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break

        
# Save in list
x_coord = []
y_coord = []

for i in range(len(coord)):
    x_cord = []
    y_cord = []
    for k in range(len(coord[i])):
        x_cord.append(coord[i][k][1])
        y_cord.append(coord[i][k][0])
    x_coord.append(x_cord)
    y_coord.append(y_cord)
            
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

# Animate

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation 
import numpy as np

figsize = np.array([1, 1/1.618]) *9. / 2.5
    
fig, ax =plt.subplots(figsize=figsize, dpi=300)
ax.axis([-5,w,h,0])
plt.gca().set_aspect('equal', adjustable='box')
original, = plt.plot([],[], ls="", marker=".", color="g" )#, markersize=0.1)
new_frame, = plt.plot([],[], ls="", marker=".", color="r")#, markersize=0.1)


def update(i):
    original.set_data(x_coord[0][:],y_coord[0][:])
    new_frame.set_data(x_coord[i][:],y_coord[i][:])
    return original, new_frame

anim = FuncAnimation(fig, update, frames=len(x_coord)-1, repeat=True)

# Plot single frame

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as signal

figsize = np.array([1, 1/1.618]) * 11/2.5
fig, ax = plt.subplots(figsize=figsize, dpi=300)
ax.axis([0,w,h,0])
plt.gca().set_aspect('equal', adjustable='box')
ax.plot(x_coord[0][:],y_coord[0][:],ls="", marker=",", color="g")
ax.plot(x_coord[175][:],y_coord[175][:],ls="", marker=",", color="r")
#fig.savefig("Figures/EdgeDetectionModeShape.pdf", dpi=300, figsize=figsize)